# Create ENCODE datasets

This script requires two inputs:

- `data/cleaned_encode_list.xlsx` - a curated Excel file listing each unique analysis using stranded or unstranded bigWig files in the ENCODE dataset
-  `data/md5list.txt` - a text file listing the md5 sum and the full path for each bigWig file on the server

This script will DELETE all existing encode datasets from the database, then load new ones, mapping the dataset to the location on the server.

In [77]:
from django.conf import settings

from collections import defaultdict
import json
import os
import pandas as pd
import numpy as np

from analysis import models

Delete old encode datasets:

In [78]:
models.EncodeDataset.objects.all().delete()

(9004,
 {'analysis.AnalysisDatasets': 0,
  'analysis.EncodeDataset': 4502,
  'analysis.GenomicDataset': 4502,
  'analysis.GenomicDataset_borrowers': 0})

Load new datasets from Excel list:

In [79]:
fn = os.path.abspath('./data/cleaned_encode_list.xlsx')
assert os.path.exists(fn)
df = pd.read_excel(fn, sheetname="Sheet1")

In [82]:
md5_fn = os.path.abspath('data/md5list.txt')
assert os.path.exists(md5_fn)

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4502 entries, 0 to 4501
Data columns (total 64 columns):
Name                    4502 non-null object
Description             0 non-null float64
plus_bigwig             685 non-null object
minus_bigwig            685 non-null object
ambig_bigwig            3817 non-null object
genome_assembly         4502 non-null object
dataType                4502 non-null object
cell                    4487 non-null object
antibody                2674 non-null object
rnaExtract              878 non-null object
phase                   128 non-null object
treatment               2672 non-null object
localization            824 non-null object
Valid                   0 non-null float64
labExpId                2620 non-null object
dccAccession            4490 non-null object
controlId               2805 non-null object
project                 4474 non-null object
ambig_md5sum            3789 non-null object
ambig_view              3817 non-null object
su

## Check max-length for each column

check db-settings to ensure our value will fit:

In [81]:
for colname in df.columns:
    series = df[colname]
    if series.dtype == np.object:
        print("{}: {}".format(colname, max([len(str(x)) for x in series.unique() if x is not np.NaN])))
    elif series.dtype == np.float64:
        print("{}: {} to {}".format(colname, series.min(), series.max()))
    else:
        print("{}: {}".format(colname, series.dtype))

Name: 64
Description: nan to nan
plus_bigwig: 69
minus_bigwig: 70
ambig_bigwig: 71
genome_assembly: 4
dataType: 10
cell: 25
antibody: 22
rnaExtract: 12
phase: 4
treatment: 24
localization: 11
Valid: nan to nan
labExpId: 27
dccAccession: 16
controlId: 29
project: 8
ambig_md5sum: 32
ambig_view: 19
subId: 1122.0 to 7873.0
protocol: 9
replicate: 1.0 to 14.0
lab: 11
type: 8
ambig_tableName: 62
geoSampleAccession: 10
setType: 5
dateUnrestricted: 19
dataVersion: 23
ambig_size: 5
composite: 23
grant: 9
dateSubmitted: datetime64[ns]
origAssembly: 4
labVersion: 150
control: 18
dateResubmitted: 19
plus_md5sum: 32
readType: 6
plus_tableName: 62
plus_view: 13
minus_md5sum: 32
minus_tableName: 63
minus_size: 5
plus_size: 5
donorId: 12
bioRep: 9
minus_view: 14
seqPlatform: 19
spikeInPool: 6
sex: 1
mapAlgorithm: 6
platform: 8
submittedDataVersion: 183
insertLength: 4
expId: 28.0 to 997.0
labProtocolId: 9
uniqueness: 25
sourceObj: 107
softwareVersion: 15
age: 14
strain: 17
tissueSourceType: 10


## Create list with local file paths

We have a list of bigWig files and md5 values for all files. We now need to map these files to our mapping in this Excel crosswalk:

In [83]:
encode_root = "/apps/encodeTracks/"

def getFileLocationDict(fn):
    cw = defaultdict(dict)
    
    with open(fn, 'r') as f:
        lines = f.readlines()
    
    lines = [ln.split() for ln in lines]
    
    for md5, fn in lines:
        name = os.path.basename(fn)
        path = fn.replace(encode_root, '')  # remove root
        cw[name][md5] = path
    
    return cw

cw = getFileLocationDict(md5_fn)

In [84]:
cw['wgEncodeCshlShortRnaSeqImr90CytosolShorttotalMinusRep2.bigWig']

{'d7c45b1d84503fb5b978f5778993e96c': 'hg19/wgEncodeCshlShortRnaSeq/wgEncodeCshlShortRnaSeqImr90CytosolShorttotalMinusRep2.bigWig'}

Now, create method for mapping a filename and optionally an MD5 to a file on our filesystem:

In [85]:
def getMatchingPath(name, md5=None):
    files = cw[name]
    
    # first, see if we're missing a name in the crosswalk
    if len(files) == 0:
        return print('Missing name: {}'.format(name))
    
    # first, try to get using MD5        
    try:
        return files[md5]
    except Exception:
        if md5:
            # next, see if we're not matching an MD5
            print('Unmatched MD5: {} - our MD5: {}, from db: {}'.format(
                name, md5, '|'.join(files.keys())
            ))        
    
    # next, if no MD5 but only one name, use this value
    if len(files) == 1 and md5 is None:
        return list(files.values())[0]
    
    print('Unmatched: {} {}'.format(name, md5))  

Set filename paths:

In [86]:
def func(d, fld, md5fld):
    name = d[fld]
    md5 = d[md5fld]
    if md5 is np.NaN:
        md5 = None
    if name is not np.NaN:
        path = getMatchingPath(name, md5)
        if path:
            return path
    return ''

df['_plus_bigwig_fn'] = df.apply(func, axis=1, args=('plus_bigwig', 'plus_md5sum'))
df['_minus_bigwig_fn'] = df.apply(func, axis=1, args=('minus_bigwig', 'minus_md5sum'))
df['_ambig_bigwig_fn'] = df.apply(func, axis=1, args=('ambig_bigwig', 'ambig_md5sum'))

Missing name: wgEncodeCaltechTfbsC2c12InputFCntrl36bE2p60hPcr1xSigRep1.bigWig
Missing name: wgEncodeSydhRnaSeqMelRibozerogR2x101dSigRep2V2.document.tgz
Missing name: wgEncodeCaltechTfbsC2c12InputFCntrl50bPcr1xSigRep2.bigWig
Missing name: wgEncodeCaltechTfbsC2c12InputFCntrl50bPcr1xSigRep3.bigWig


### Cleanup genome type:

In [87]:
def func(d):
    if d.genome_assembly == 'hg19':
        return models.HG19
    elif d.genome_assembly == 'mm9':
        return models.MM9
    else:
        raise ValueError()

df['_genome_assembly'] = df.apply(func, axis=1)

### Cleanup cases where text may be blank:

In [88]:
# If commented, this means the field must not be blank or 
# else an error should be thrown, or is handled elsewhere.

fields = [
    #'Name',
    'Description',
    #'plus_bigwig',
    #'minus_bigwig',
    #'ambig_bigwig',
    #'genome_assembly',
    #'dataType',
    'cell',
    'antibody',
    'rnaExtract',
    'phase',
    'treatment',
    'localization',
    'labExpId',
    'dccAccession',
    'controlId',
    'project'
    'labExpId',
    'dccAccession',
    'controlId',
    'project',
    'ambig_md5sum',
    'ambig_view',
    #'subId',
    'protocol',
    #'replicate',
    'lab',
    #'type',
    'ambig_tableName',
    'geoSampleAccession',
    'setType',
    #'dateUnrestricted',
    #'dataVersion',
    'ambig_size',
    'composite',
    #'grant',
    #'dateSubmitted',
    'origAssembly',
    'labVersion',
    'control',
    #'dateResubmitted',
    'plus_md5sum',
    'readType',
    'plus_tableName',
    'plus_view',
    'minus_md5sum',
    'minus_tableName',
    'minus_size',
    'plus_size',
    'donorId',
    'bioRep',
    'minus_view',
    'seqPlatform',
    'spikeInPool',
    'sex',
    'mapAlgorithm',
    'platform',
    'submittedDataVersion',
    #'insertLength',
    #'expId',
    'labProtocolId',
    'uniqueness',
    'sourceObj',
    'softwareVersion',
    'age',
    'strain',
    'tissueSourceType',

]
for fld in fields:
    if fld not in df.columns:
        continue
    df[fld].fillna(value='', inplace=True)

### Clean dates

In [89]:
def func(d, fld):
    val = d[fld]
    if val is not np.NaN and val is not pd.NaT:
        try:
            return val.toordinal()
        except AttributeError:
            # invalid date
            print("Invalid date: {}".format(val))
    return None

df['dateUnrestricted'] = df.apply(func, axis=1, args=('dateUnrestricted', ))
df['dateSubmitted'] = df.apply(func, axis=1, args=('dateSubmitted', ))
df['dateResubmitted'] = df.apply(func, axis=1, args=('dateResubmitted', ))

Invalid date: 2012-11-31


### set NaN to None (for numeric fields)

In [90]:
fields = [
    'subId',
    'replicate',
    'dataVersion',
    'insertLength',
    'expId',
    'dateUnrestricted',
    'dateSubmitted',
    'dateResubmitted',
]
for fld in fields:
    df[fld] = df[fld].where(pd.notnull(df[fld]), other=None)    

In [91]:
# coerce to string (some datetimes mixed in)
df['dataVersion'] = df.dataVersion.astype(str)

### Set extra content

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4502 entries, 0 to 4501
Data columns (total 68 columns):
Name                    4502 non-null object
Description             4502 non-null object
plus_bigwig             685 non-null object
minus_bigwig            685 non-null object
ambig_bigwig            3817 non-null object
genome_assembly         4502 non-null object
dataType                4502 non-null object
cell                    4502 non-null object
antibody                4502 non-null object
rnaExtract              4502 non-null object
phase                   4502 non-null object
treatment               4502 non-null object
localization            4502 non-null object
Valid                   0 non-null float64
labExpId                4502 non-null object
dccAccession            4502 non-null object
controlId               4502 non-null object
project                 4502 non-null object
ambig_md5sum            4502 non-null object
ambig_view              4502 non-null obje

In [93]:
extra_content_fields = [
    'labExpId',
    'dccAccession',
    'controlId',
    'project',
    'ambig_md5sum',
    'ambig_view',
    'subId',
    'protocol',
    'replicate',
    'lab',
    'type',
    'ambig_tableName',
    'geoSampleAccession',
    'setType',
    'dateUnrestricted',
    'dataVersion',
    'ambig_size',
    'composite',
    'grant',
    'dateSubmitted',
    'origAssembly',
    'labVersion',
    'control',
    'dateResubmitted',
    'plus_md5sum',
    'readType',
    'plus_tableName',
    'plus_view',
    'minus_md5sum',
    'minus_tableName',
    'minus_size',
    'plus_size',
    'donorId',
    'bioRep',
    'minus_view',
    'seqPlatform',
    'spikeInPool',
    'sex',
    'mapAlgorithm',
    'platform',
    'submittedDataVersion',
    'insertLength',
    'expId',
    'labProtocolId',
    'uniqueness',
    'sourceObj',
    'softwareVersion',
    'age',
    'strain',
    'tissueSourceType',
]

dtype_datetime = np.dtype('datetime64[ns]')

def getExtraContent(d):
    content = {}
    for fld in extra_content_fields:
        val = d[fld]
        if val and val is not pd.NaT:
            if df[fld].dtype is dtype_datetime:
                content[fld] = d[fld].toordinal()
            elif val is not np.NaN:
                content[fld] = d[fld]
    return content    

### Create objects!

In [94]:
for i, d in df.iterrows():
    extra_content = getExtraContent(d)
    object_ = models.EncodeDataset(
        name=d.Name,
        public=True,
        genome_assembly=d._genome_assembly,
        data_type=d.dataType,
        cell_type=d.cell,
        antibody=d.antibody,
        rna_extract=d.rnaExtract,
        treatment=d.treatment,
        phase=d.phase,
        localization=d.localization,    
        extra_content = extra_content   
    )
    
    if len(d._ambig_bigwig_fn) > 0:
        object_.data_ambiguous.name = d._ambig_bigwig_fn
    
    if len(d._plus_bigwig_fn) > 0:
        object_.data_plus.name = d._plus_bigwig_fn
    
    if len(d._minus_bigwig_fn) > 0:
        object_.data_minus.name = d._minus_bigwig_fn
    
    object_.save()